## Play around datasets and create one with hierarchical prior


In [1]:
import os
import sys
from tensorflow.compat.v1 import gfile

sys.path.append(os.path.abspath("/home/mwu34/disentanglement_lib"))
import numpy as np
import pandas as pd

DSPRITES_PATH = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites",
    "dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz"
)

In [7]:
with gfile.Open(DSPRITES_PATH, "rb") as data_file:
    # Data was saved originally using python2, so we need to set the encoding.
    data = np.load(data_file, encoding="latin1", allow_pickle=True)
    images = np.array(data["imgs"])  # has shape 737280 * 64 * 64
    factor_sizes = np.array(data["metadata"][()]["latents_sizes"], dtype=np.int64)

In [14]:
from disentanglement_lib.data.ground_truth.util import CorrelatedSplitDiscreteStateSpace


correlated_space = CorrelatedSplitDiscreteStateSpace(factor_sizes, [3, 4])

RuntimeError: Required bindings for `correlation_details` not provided in config: ['corr_indices', 'corr_type']

In [39]:
# trying to get mapping from specific parameter combinations to model num
from disentanglement_lib.config.unsupervised_study_v1.sweep import get_config
import pandas as pd

configs = get_config()
df = pd.DataFrame(configs)

In [3]:
len(df)

54

In [6]:
# trying to get mapping from specific parameter combinations to model num
# from disentanglement_lib.config.unsupervised_study_v1.sweep import get_config
from disentanglement_lib.config.double_descent_study.sweep import get_config
import pandas as pd

configs = get_config()
print(configs[12])  # give the config of model 12

df = pd.DataFrame(configs)

# Get the beta_tc_vae models using dsprites and a beta param of 4.
model = df['model.name'] == "beta_vae"
dataset = df['dataset.name'] == "dsprites_full"
beta = df["vae.beta"] == 1.0

# Give all the possible model configs unused parameters excluded (e.g., annealed_vae.gamma)
# print(df[model & dataset & beta].dropna(axis=1))

# Give the list of model ids with this configuration
df[model & dataset].head(200)


{'correlation_details.corr_indices': [3, 4], 'dataset.name': 'dsprites_full', 'correlation_hyperparameter.line_width': 10.0, 'correlation_details.corr_type': 'line', 'encoder.encoder_fn': '@conv_encoder', 'decoder.decoder_fn': '@deconv_decoder', 'correlation.active_correlation': True, 'model.name': 'beta_vae', 'vae.beta': 0.25, 'model.model': '@vae()', 'conv_encoder.num_parameters_scale': 0.0625, 'deconv_decoder.num_parameters_scale': 0.0625, 'model.random_seed': 0}


,correlation_details.corr_indices,dataset.name,correlation_hyperparameter.line_width,correlation_details.corr_type,encoder.encoder_fn,decoder.decoder_fn,correlation.active_correlation,model.name,vae.beta,model.model,conv_encoder.num_parameters_scale,deconv_decoder.num_parameters_scale,model.random_seed
0,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),0.0625,0.0625,0
1,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),0.1250,0.1250,0
2,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),0.2500,0.2500,0
3,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),0.5000,0.5000,0
4,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),1.0000,1.0000,0
5,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.001,@vae(),4.0000,4.0000,0
6,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.0625,0.0625,0
7,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.1250,0.1250,0
8,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.2500,0.2500,0
9,"[3, 4]",dsprites_full,10.0,line,@conv_encoder,@deconv_decoder,True,beta_vae,0.125,@vae(),0.5000,0.5000,0


In [24]:
df['dataset.name'].value_counts()

shapes3d         2160
mpi3d_real        720
dsprites_full     720
Name: dataset.name, dtype: int64